In [6]:
import pandas as pd
import os
from glob import glob

# File paths
clinical_file = r"D:\mlpr data\Glioblastoma-ML-model\UPENN-GBM_clinical_info_v2.1.csv"
radiomics_folder = r"D:\mlpr data\radiomic_features_CaPTk"

# Load clinical data
clinical_df = pd.read_csv(clinical_file)
clinical_df.rename(columns={"ID": "PatientID"}, inplace=True)  # Standardizing ID column name

# Load all radiomic CSVs and merge horizontally on PatientID
radiomic_files = glob(os.path.join(radiomics_folder, "*.csv"))

# Initialize empty dataframe for radiomics
radiomics_df = pd.DataFrame()

for file in radiomic_files:
    df = pd.read_csv(file)
    df.rename(columns={"SubjectID": "PatientID"}, inplace=True)  # Standardizing ID column name
    
    # Merge radiomics files horizontally
    if radiomics_df.empty:
        radiomics_df = df
    else:
        radiomics_df = pd.merge(radiomics_df, df, on="PatientID", how="outer")

# Merge clinical data with radiomics data
merged_df = pd.merge(clinical_df, radiomics_df, on="PatientID", how="outer")

# Save final merged dataset
output_file = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
merged_df.to_csv(output_file, index=False)

print(f"Merged dataset saved at {output_file}")


Merged dataset saved at D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (forcing errors='coerce' turns non-numeric values into NaN)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED"])  # Drop ID and target
y = df["Survival_from_surgery_days_UPDATED"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoder for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

C:\Users\asp61\AppData\Local\Temp\ipykernel_26020\2633508008.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)  # Use all cores
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


Mean Absolute Error: 342.39
R² Score: 0.0764


In [4]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, explained_variance_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Root Mean Squared Error
evs = explained_variance_score(y_test, y_pred)  # Explained Variance Score

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Explained Variance Score: {evs:.4f}")


Mean Absolute Error: 342.39
R² Score: 0.0764
Root Mean Squared Error (RMSE): 475.31
Explained Variance Score: 0.0794


## Classifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (handling non-numeric values)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# **Convert Survival Days into Categories (Example Binning)**
# You can modify bins as per your requirement
bins = [0, 100, 365, np.inf]  # <100 days, 100-365 days, >365 days

labels = [0, 1, 2]  # Class labels
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoders for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


C:\Users\asp61\AppData\Local\Temp\ipykernel_26020\671799816.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Accuracy: 0.30
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.39      0.43        33
           1       0.30      0.19      0.23        32
           2       0.24      0.31      0.27        32
           3       0.25      0.31      0.28        32

    accuracy                           0.30       129
   macro avg       0.32      0.30      0.30       129
weighted avg       0.32      0.30      0.30       129



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Using quantiles for more balanced classes
q_25 = df["Survival_from_surgery_days_UPDATED"].quantile(0.25)
q_50 = df["Survival_from_surgery_days_UPDATED"].quantile(0.50)
q_75 = df["Survival_from_surgery_days_UPDATED"].quantile(0.75)

# Create classes based on quartiles
bins = [0, q_25, q_50, q_75, np.inf]
labels = [0, 1, 2, 3]  # 4 classes based on quartiles
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Print the quantile values and class distribution
print(f"Q1 (25%): {q_25:.2f} days")
print(f"Q2 (50%): {q_50:.2f} days")
print(f"Q3 (75%): {q_75:.2f} days")
print("\nClass distribution:")
print(df["Survival_Category"].value_counts())

# Separate features and target
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Convert all categorical columns to string type
for col in categorical_cols:
    X[col] = X[col].astype(str)

# Create dummy variables for categorical features
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Fill missing values with median
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Standardize features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"Cross-validation accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f"Test accuracy: {accuracy:.4f}")
    print("Classification Report:\n", report)
    
    # Feature importance
    if hasattr(model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Feature': X.columns,
            'Importance': model.feature_importances_
        })
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        print("\nTop 10 important features:")
        print(feature_importance.head(10))
    
    return model, accuracy

# Try different models with optimized parameters
print("\n===== Random Forest =====")
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)
rf_model, rf_accuracy = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

print("\n===== Gradient Boosting =====")
gb_model = GradientBoostingClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_model, gb_accuracy = evaluate_model(gb_model, X_train, X_test, y_train, y_test)

# Use the better model for final predictions
best_model = rf_model if rf_accuracy >= gb_accuracy else gb_model
print(f"\nBest model: {'Random Forest' if rf_accuracy >= gb_accuracy else 'Gradient Boosting'}")

C:\Users\asp61\AppData\Local\Temp\ipykernel_26020\1185912683.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Q1 (25%): 167.00 days
Q2 (50%): 376.00 days
Q3 (75%): 618.25 days

Class distribution:
Survival_Category
0    162
1    162
3    161
2    159
Name: count, dtype: int64

===== Random Forest =====
Cross-validation accuracy: 0.3417 ± 0.0207
Test accuracy: 0.3256
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.45      0.47        33
           1       0.27      0.22      0.24        32
           2       0.29      0.34      0.31        32
           3       0.26      0.28      0.27        32

    accuracy                           0.33       129
   macro avg       0.33      0.32      0.32       129
weighted avg       0.33      0.33      0.33       129


Top 10 important features:
                                      Feature  Importance
0                           Age_at_scan_years    0.011898
1                                PsP_TP_score    0.005387
9506                                 Gender_M    0.003833
9509                  

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (handling non-numeric values)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# **Convert Survival Days into Categories (Example Binning)**
# You can modify bins as per your requirement
bins = [0, 100, 365, np.inf]  # <100 days, 100-365 days, >365 days
labels = [0, 1, 2]  # Class labels
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoders for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)
